In [1]:
#Part 2: Intro to Federated Learning
import torch
from torch import nn
from torch import optim
import syft as sy

hook = sy.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\ProgramData\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
#Section 2.1 - A Toy Federated Learning Example

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
#model = nn.Linear(2,1)

#def train1():
    # Training Logic
 #   opt = optim.SGD(params=model.parameters(),lr=0.1)
  #  for iter in range(20):

        # 1) erase previous gradients (if they exist)
 #       opt.zero_grad()

        # 2) make a prediction
  #      pred = model(data)

        # 3) calculate how much we missed
   #     loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
    #    loss.backward()

        # 5) change those weights
     #   opt.step()

        # 6) print our progress
      #  print(loss.data)
    
#train1()

In [3]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

# get pointers to training data on each worker by
# sending some training data to bob and alice
#split the data set right in half
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

#send data set to bob and alice
data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

#tensor for data and target on bob and alice

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [4]:
def train2():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging
train2()

tensor(1.2481, requires_grad=True)
tensor(0.0721, requires_grad=True)
tensor(0.4442, requires_grad=True)
tensor(0.1362, requires_grad=True)
tensor(0.2614, requires_grad=True)
tensor(0.0872, requires_grad=True)
tensor(0.1615, requires_grad=True)
tensor(0.0534, requires_grad=True)
tensor(0.1010, requires_grad=True)
tensor(0.0330, requires_grad=True)
tensor(0.0639, requires_grad=True)
tensor(0.0208, requires_grad=True)
tensor(0.0409, requires_grad=True)
tensor(0.0133, requires_grad=True)
tensor(0.0265, requires_grad=True)
tensor(0.0087, requires_grad=True)
tensor(0.0174, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0116, requires_grad=True)
tensor(0.0039, requires_grad=True)
